In [1]:
import openai 
import json
import argparse
import tqdm
import time
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import pandas as pd
from dotenv import load_dotenv 

load_dotenv()

def calculate_average(scores, threshold):
    # Attempt to convert scores to floats, ignoring invalid entries
    valid_scores = []
    invalid_count = 0
    
    for score in scores:
        try:
            valid_scores.append(float(score))
        except ValueError:
            invalid_count += 1
            
    # If more than two invalid entries, return average as 0
    if invalid_count > threshold:
        return 0
    else:
        return sum(valid_scores) / len(valid_scores) if valid_scores else 0

prompt_filepath ="./prompts/category/corr_detailed.txt"




In [2]:

with open("data/gen_raw_source.json", 'r', encoding='utf-8') as file:
    data_json = json.load(file)
data = pd.DataFrame(data_json)
prompt = open(prompt_filepath).read()
data_type = 'ct'
llm = ChatOpenAI(model='gpt-3.5-turbo-0125')
prompt = ChatPromptTemplate.from_template(prompt) 
cur_prompt = prompt
parser = StrOutputParser()
Raw_source = data["raw_source"]
input_prompt_text = "\n\nSource Product: {Raw_source}\n" \
                    "Category: {Product_category}\n" \
                    "\nEvaluation Form (scores ONLY):"\
                    "\nCorrectness:"

if data_type == 'pn':
    chain = {"Raw_source": RunnablePassthrough(), "Product_name": RunnablePassthrough()}| prompt | llm | parser
    Product_name = data["product_title"]
    product_info = list(zip(Raw_source, Product_name))
elif data_type == 'cp':
    chain = {"Raw_source": RunnablePassthrough(), "Catch_phrase": RunnablePassthrough()}| prompt | llm | parser
    Catch_phrase = data["detailed_description"]
    product_info = list(zip(Raw_source, Catch_phrase))
elif data_type == 'rk':
    chain = {"Raw_source": RunnablePassthrough(), "Recommended_keywords": RunnablePassthrough()}| prompt | llm | parser
    Recommended_keywords = data["related_search_keywords"]
    product_info = list(zip(Raw_source, Recommended_keywords))
elif data_type == 'ct':
        chain = {"Raw_source": RunnablePassthrough(), "Product_category": RunnablePassthrough()}| prompt | llm | parser
        Product_category = data["product_category"]
        product_info = list(zip(Raw_source, Product_category))
        print(product_info)
print(product_info)



[('상해 광저우 항저우 북경 심천 우한 청두 영화표권 UME 완다 진이 CGV 타오표표 고양이 눈', '스낵과 음료'), ('부직포 2023 크리스마스 문 헬멧 장식 화환 손수 작업 DIY 재료 세트 어린이 장식 장면 배치', '모형 장난감/애니메이션/주변기기'), ('10인치 abike 경량 인력 접이식 자전거 풋페달 8인치 미니 접이식 자전거 초소형 풋페달 무료배송', '야외 스포츠'), ('스테인레스 스틸 우유 냄비 스프 냄비 두꺼운 면을 끓이는 작은 우유 냄비 미니 냄비 라면을 끓이는 보조 식기용 냄비 전기 인덕션 가스 호환 가능', '가정 생활'), ('진가 손상 보정 자동 접착 슬라이딩 양 가죽 패치 패치 가죽 의류 가죽 소품 소파 보강 구멍 수리', '가정 생활'), ('광장댄스 신상 의복 여성 가을 시즌 장식 금 섬유 코튼 춤 무대 공연 춤 복장 드레스 세트', '여성 의류/여성 신발/여성 부티크'), ('베이징 상해 청다오 타오피아오 특가 영화표 만다 CGV 대지 금이 UME 보나 UA 수도 워메이', '스낵과 음료'), ('미국 Swiffer 보레아킹 스팀 모핑 클리너 더스트 패드 교체 세트입니다.', '여성 의류/여성 신밠/여성 부티크'), ('야외 플리스 여성 가디건 더블 페이스 플리스 스탠드 칼라 자켓 소프트 쉘 스톰 재킷 안감 두껍게 보온 모피 외투', '가정 생활'), ('귀여운 멜로디 분홍색 침실 장식 벽시계 멜로디 쥐 퍼즐 시계 소녀 마음 방 장식품', '가정 생활'), ('산지 자전거 실내 주차대 거치대 벽걸이 후크 전시대 세로로 두 대의 자전거 지지대', '야외 스포츠'), ('느린 탄성 스쿼시 부드러운 향기가 나는 인조 빵 대형 펜던트 가짜 음식 모형 장식 발산 스트레스 해소 장난감', '모형 장난감/애니메이션/주변기기'), ('스마트 스터디 기판 컴퓨터 학생 1학년부터 고등학교 초등학교 교과서 동기화된 영어 읽기 기계 어린이', '3C 디지털/휴대전화/스마트 기기'), ('덕이 캠프 전술 스틸 와이어 페이스 마스크 얼굴과

In [3]:
chain

{
  Raw_source: RunnablePassthrough(),
  Product_category: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=[], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="You are provided with the name of a Korean product(Source Product) and a category designation for that product (Category). Your task is to evaluate whether the assigned category is appropriate for the product based on the given criteria.\n\nPlease read these instructions carefully and refer back to them as necessary while performing the evaluation.\n\nEvaluation Criteria:\n\nCorrectness (1-5): Assess whether the category is suitable for the product. Consider that the product description might be ambiguous, making the categorization challenging. Categories to choose from include:\n\n가정 생활 (Home Living)\n여성 의류/여성 신발/여성 부티크 (Women's Clothing/Shoes/Boutique)\n남성 의류/남성 신발/남성 부티크 (Men's Clothing/Shoes/Boutique)\n가방/액세서리 (Bags/Accessories)\n야외 스포츠 (Outdoor Sports)\n3C 디지털/휴대전화/스마트 기기

In [4]:
print("Chain Input Example:", chain.invoke(product_info))

Chain Input Example: Unfortunately, the Source Product and Category were not provided. Please input the necessary information for evaluation.


In [45]:
score = chain.batch(product_info, config={"max_concurrency": 15})

In [46]:
for d in score:
    print("\n",score)


 ['Unfortunately, the evaluation form cannot be completed without the specific source product and category. Please provide the necessary information for the evaluation to be conducted.', 'Unfortunately, since the Source Product and Product Category are not provided, I am unable to evaluate the appropriateness of the category for the product. Please provide the necessary information so that I can assist you in evaluating the categorization.', 'Unfortunately, the Source Product and Category are missing. Could you please provide that information so I can evaluate it for you? Thank you!', 'Unfortunately, I cannot perform the evaluation without the specific source product and category provided. Please input the details, and I will be able to assist you in evaluating the appropriateness of the category for the product.', 'Unfortunately, I cannot provide an evaluation as the source product and category are missing. Please provide the necessary details for evaluation.', 'Unfortunately, the ev

In [44]:
score

['Unfortunately, the evaluation cannot be completed as the "Source Product" and "Product Category" are missing. Please provide the necessary information to proceed with the evaluation.',
 'Unfortunately, I cannot provide an evaluation without the specific source product and category. Once you provide that information, I will be able to assist with the evaluation.',
 'Unfortunately, the evaluation cannot be completed as the Source Product and Product Category are missing. Please provide the necessary information to proceed.',
 "Unfortunately, as the Source Product and Category are not provided, I'm unable to evaluate the appropriateness of the category for the product. Please provide the necessary information so I can complete the evaluation.",
 'Unfortunately, the evaluation form cannot be completed without knowing the actual source product and category. Please provide the specific details so that I can assist you in evaluating the appropriateness of the category for the product.',
 'U